## imports

In [1]:
from os import environ as ENV
from dotenv import load_dotenv
from pymssql import connect
import pandas as pd
from boto3 import client

## load data in

In [2]:
load_dotenv()

True

In [4]:
def get_db_connection(config: dict) -> connect:
    """Returns database connection."""

    return connect(
        server=config["DB_HOST"],
        port=config["DB_PORT"],
        user=config["DB_USER"],
        database=config["DB_NAME"],
        password=config["DB_PASSWORD"],
        as_dict=True
    )

In [20]:
connection = get_db_connection(ENV)
def get_df(conn: connect) -> pd.DataFrame:
    """Returns a Dataframe of method data from database."""

    query = """ 
            SELECT *
            FROM s_beta.recording AS r

            """
    
    with conn.cursor() as cur:
        cur.execute(query)
        rows = cur.fetchall()


    
    return rows
data = pd.DataFrame(get_df(connection))
print([pd.DataFrame(data)['recording_taken']])

[      recording_id  plant_id     recording_taken        last_watered  \
0                1         0 2024-04-16 15:42:01 2024-04-16 14:03:04   
1                2         1 2024-04-16 15:41:51 2024-04-16 13:54:32   
2                3         2 2024-04-16 15:41:49 2024-04-16 14:10:54   
3                4         3 2024-04-16 15:42:00 2024-04-16 14:50:16   
4                5         4 2024-04-16 15:41:58 2024-04-16 13:16:25   
...            ...       ...                 ...                 ...   
1281          1282        44 2024-04-16 16:17:17 2024-04-16 13:32:40   
1282          1283        46 2024-04-16 16:17:04 2024-04-16 14:12:39   
1283          1284        47 2024-04-16 16:17:15 2024-04-16 13:30:18   
1284          1285        48 2024-04-16 16:17:17 2024-04-16 14:31:09   
1285          1286        49 2024-04-16 16:17:10 2024-04-16 14:02:15   

     soil_moisture temperature  image_id  botanist_id  
0          94.2520     13.2095         1            1  
1          93.6619    

In [16]:
data = get_df(connection)

In [10]:
data

,plant_id,plant_name,scientific_name,soil_moisture,temperature
0,0,Epipremnum Aureum,Epipremnum aureum,94.2520,13.2095
1,1,Venus flytrap,None,93.6619,12.0518
2,2,Corpse flower,None,95.0359,9.1838
3,3,Rafflesia arnoldii,None,97.0180,10.0402
4,4,Black bat flower,None,91.3188,11.3533
...,...,...,...,...,...
1281,44,Araucaria Heterophylla,Araucaria heterophylla,90.5837,9.5606
1282,46,Medinilla Magnifica,Medinilla magnifica,93.1255,13.3093
1283,47,Calliandra Haematocephala,Calliandra haematocephala,85.6893,13.3377
1284,48,Zamioculcas Zamiifolia,Zamioculcas zamiifolia,93.7807,14.8889


## transform data

In [16]:
data.groupby(["plant_id", "plant_name", "scientific_name"],
             as_index=False).agg(["mean", "std", "min", "max"],)

plant_id                 plant_name                      scientific_name  \
                                                                              
0         0          Epipremnum Aureum                    Epipremnum aureum   
1         5              Pitcher plant                 Sarracenia catesbaei   
2         6               Wollemi pine                     Wollemia nobilis   
3         8           Bird of paradise  Heliconia schiedeana 'Fire and Ice'   
4         9                     Cactus                 Pereskia grandifolia   
5        11      Asclepias Curassavica                Asclepias curassavica   
6        14        Colocasia Esculenta                  Colocasia esculenta   
7        16      Euphorbia Cotinifolia                Euphorbia cotinifolia   
8        17            Ipomoea Batatas                      Ipomoea batatas   
9        19                Musa Basjoo                          Musa basjoo   
10       20           Salvia Splendens                     Salvia splendens   
11       21                  Anthurium                 Anthurium andraeanum   
12       22           Bird of Paradise  Heliconia schiedeana 'Fire and Ice'   
13       23        Cordyline Fruticosa                  Cordyline fruticosa   
14       24                      Ficus                         Ficus carica   
15       26      Dieffenbachia Seguine                Dieffenbachia seguine   
16       27              Spathiphyllum                Spathiphyllum (group)   
17       28                     Croton                  Codiaeum variegatum   
18       29                  Aloe Vera                            Aloe vera   
19       30             Ficus Elastica                       Ficus elastica   
20       31    Sansevieria Trifasciata              Sansevieria trifasciata   
21       32    Philodendron Hederaceum              Philodendron hederaceum   
22       33      Schefflera Arboricola                Schefflera arboricola   
23       34       Aglaonema Commutatum                 Aglaonema commutatum   
24       35         Monstera Deliciosa                   Monstera deliciosa   
25       36         Tacca Integrifolia                   Tacca integrifolia   
26       38       Saintpaulia Ionantha                 Saintpaulia ionantha   
27       39                 Gaillardia                Gaillardia aestivalis   
28       40                  Amaryllis                  Hippeastrum (group)   
29       41           Caladium Bicolor                     Caladium bicolor   
30       42       Chlorophytum Comosum      Chlorophytum comosum 'Vittatum'   
31       44     Araucaria Heterophylla               Araucaria heterophylla   
32       45                    Begonia                  Begonia 'Art Hodes'   
33       46        Medinilla Magnifica                  Medinilla magnifica   
34       47  Calliandra Haematocephala            Calliandra haematocephala   
35       48     Zamioculcas Zamiifolia               Zamioculcas zamiifolia   
36       49             Crassula Ovata                       Crassula ovata   

   soil_moisture                               temperature             \
            mean        std       min      max        mean        std   
0      93.013725   0.514017   92.2081  94.2520   13.701857   2.379130   
1      96.209622   0.506180   95.4097  97.4099   12.487789   4.633637   
2      91.801996   0.503721   91.0140  92.9740     11.3168   1.731956   
3      90.437371   0.527593   89.6138  91.7218   11.634261   0.492161   
4      93.603889   0.512188   92.7904  94.8320   10.779226   0.890832   
5      91.820974   0.491859   91.0206  92.9970   13.859922  19.086600   
6      94.031622   0.501265   93.2657  95.2560   13.354689   1.076748   
7      88.625107   0.771758   87.4441  90.4646    13.29723   0.036386   
8      92.963623   0.510482   92.1684  94.2281   10.303438   2.496179   
9        91.1168   0.493620   90.3522  92.3086   14.593429   3.148154   
10     95.698835   0.527919   94.8657  96.922

In [12]:
data.groupby(["plant_id", "plant_name", "scientific_name"],
             as_index=False).mean()

,plant_id,plant_name,scientific_name,soil_moisture,temperature
0,0,Epipremnum Aureum,Epipremnum aureum,93.013725,13.701857
1,5,Pitcher plant,Sarracenia catesbaei,96.209622,12.487789
2,6,Wollemi pine,Wollemia nobilis,91.801996,11.3168
3,8,Bird of paradise,Heliconia schiedeana 'Fire and Ice',90.437371,11.634261
4,9,Cactus,Pereskia grandifolia,93.603889,10.779226
5,11,Asclepias Curassavica,Asclepias curassavica,91.820974,13.859922
6,14,Colocasia Esculenta,Colocasia esculenta,94.031622,13.354689
7,16,Euphorbia Cotinifolia,Euphorbia cotinifolia,88.625107,13.29723
8,17,Ipomoea Batatas,Ipomoea batatas,92.963623,10.303438
9,19,Musa Basjoo,Musa basjoo,91.1168,14.593429


In [13]:
data.groupby(["plant_id", "plant_name", "scientific_name"],
             as_index=False).std()

,plant_id,plant_name,scientific_name,soil_moisture,temperature
0,0,Epipremnum Aureum,Epipremnum aureum,0.514017,2.379130
1,5,Pitcher plant,Sarracenia catesbaei,0.506180,4.633637
2,6,Wollemi pine,Wollemia nobilis,0.503721,1.731956
3,8,Bird of paradise,Heliconia schiedeana 'Fire and Ice',0.527593,0.492161
4,9,Cactus,Pereskia grandifolia,0.512188,0.890832
5,11,Asclepias Curassavica,Asclepias curassavica,0.491859,19.086600
6,14,Colocasia Esculenta,Colocasia esculenta,0.501265,1.076748
7,16,Euphorbia Cotinifolia,Euphorbia cotinifolia,0.771758,0.036386
8,17,Ipomoea Batatas,Ipomoea batatas,0.510482,2.496179
9,19,Musa Basjoo,Musa basjoo,0.493620,3.148154


In [ ]:
def get_summary(df: pd.DataFrame) -> pd.DataFrame:
    """Get 1 mean per parameter per plant"""
    
    mean = df.groupby(["plant_id", "plant_name", "scientific_name"],
                      as_index=False).mean().rename(mapper={"soil_moisture": "soil_moisture_mean",
                                                            "temperature": "temperature_mean"})
    std = df.groupby(["plant_id", "plant_name", "scientific_name"],
                     as_index=False).std().rename(mapper={"soil_moisture": "soil_moisture_std",
                                                          "temperature": "temperature_mean"})
    min = df.groupby(["plant_id", "plant_name", "scientific_name"],
                     as_index=False).min()
    max = df.groupby(["plant_id", "plant_name", "scientific_name"],
                     as_index=False).max()
    df = mean.jo

## upload data

In [ ]:
S3 = client('s3',
            aws_access_key_id=ENV["AWS_ACCESS_KEY_ID"],
            aws_secret_access_key=ENV["AWS_SECRET_ACCESS_KEY"])

In [ ]:
def upload_object(client: client, file: str, bucket: str, key: str) -> None:
    """Upload file to S3 bucket.
    Returns nothing."""
    
    client.upload_file(file, bucket, key)

In [ ]:
# upload_object(mean.csv, "late-ordovician", )